In [ ]:
!pip install matplotlib librosa soundfile

In [ ]:
pip install noisereduce

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import librosa
import soundfile as sf
from scipy.signal import butter, lfilter
import noisereduce as nr

def butter_lowpass_filter(data, cutoff, fs=22050, order=5):
    """
    Apply a low-pass Butterworth filter to the data.
    """
    nyq = 0.5 * fs
    normal_cutoff = cutoff / nyq
    b, a = butter(order, normal_cutoff, btype='low', analog=False)
    y = lfilter(b, a, data)
    return y

def image_to_spectrogram(image_path):
    """
    Convert an image to a spectrogram.
    """
    try:
        img = plt.imread(image_path)
    except FileNotFoundError:
        raise Exception("Image file not found.")
    grayscale_img = np.mean(img[..., :3], axis=2)
    spectrogram_linear = librosa.db_to_amplitude((grayscale_img - grayscale_img.min()) / (grayscale_img.max() - grayscale_img.min()) * 80)
    return spectrogram_linear

def griffin_lim(spectrogram, n_iter=250, hop_length=256, win_length=1024, n_fft=None):
    """
    Perform the Griffin-Lim algorithm for phase reconstruction from a magnitude spectrogram.
    """
    if n_fft is None:
        n_fft = (spectrogram.shape[0] - 1) * 2
    phase = np.exp(2j * np.pi * np.random.rand(*spectrogram.shape))
    for i in range(n_iter):
        audio = librosa.istft(spectrogram * phase, hop_length=hop_length, win_length=win_length)
        recomplex_spec = librosa.stft(audio, n_fft=n_fft, hop_length=hop_length, win_length=win_length)
        _, phase = librosa.magphase(recomplex_spec)
    return librosa.istft(spectrogram * phase, hop_length=hop_length, win_length=win_length)

def spectrogram_to_audio(image_path, output_path, target_duration=None, n_iter=300, sample_rate=22050, hop_length=256, win_length=1024, cutoff=8000):
    """
    Convert a spectrogram image to audio.
    """
    spectrogram = image_to_spectrogram(image_path)
    if target_duration is not None:
        required_columns = int(np.ceil(target_duration * sample_rate / hop_length))
        current_columns = spectrogram.shape[1]
        padding = np.zeros((spectrogram.shape[0], max(0, required_columns - current_columns)))
        spectrogram = np.hstack((spectrogram, padding))[:, :required_columns]
    audio = griffin_lim(spectrogram, n_iter=n_iter, hop_length=hop_length, win_length=win_length)
    audio_filtered = butter_lowpass_filter(audio, cutoff=cutoff, fs=sample_rate, order=5)

    noise_clip = audio_filtered[:int(0.5 * sample_rate)]
    audio_denoised = nr.reduce_noise(y=audio_filtered, sr=sample_rate)

    sf.write(output_path, audio_denoised, sample_rate)


file_path = '/content/lp_image.jpeg'  #THIS IS FILE PATH
output_path = 'output_audio.wav'
spectrogram_to_audio(file_path, output_path, target_duration=183, n_iter=300, sample_rate=22050, hop_length=512, cutoff=10000)

